# Práctica de Aprendizaje Automático

En esta práctica vamos a estudiar unos datos utilizando métodos de aprendizaje automático.

Esta práctica esta desarrollada por Belén Gómez Arnaldo con NIA 100472037 y Luis Gómez-Manzanilla Nieto con NIA 100472006.

---
Letura de datos 
---
Vamos a empezar leyendo el archivo de entrada *wind_ava.csv* e inctroducirlo en un objeto de pandas

In [ ]:
import pandas as pd
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")

A continuación separaremos los conjuntos siendo *X* las entradas de *wind_ava* para la localización de Sotavento (13) y siendo *y* la energía eolíca que producirá el parque de Sotavento. 

In [ ]:
from sklearn.model_selection import train_test_split

#Solo nos interesan las variables que terminan en .13
x = wind_ava.filter(regex='\.13$')
y = wind_ava['energy']

Para la creación del modelo dividiremos los conjuntos que tenemos en conjuntos de entrenamiento y conjuntos de prueba.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.33, random_state=100472037)

---
EDA
---

In [3]:
X_train.head()

,p54.162.13,p55.162.13,cape.13,p59.162.13,lai_lv.13,lai_hv.13,u10n.13,v10n.13,sp.13,stl1.13,...,t2m.13,stl2.13,stl3.13,iews.13,inss.13,stl4.13,fsr.13,flsr.13,u100.13,v100.13
1435,2.393177e+06,9.466787,17.998878,1.164087e+06,2.390925,2.455061,5.935423,-0.645755,97225.342059,277.376600,...,279.117258,276.821075,279.648868,0.692994,-0.086525,281.616244,0.407292,-5.955274,7.798462,-0.893821
1623,2.469342e+06,17.103764,0.000000,8.585414e+05,2.863429,2.580754,-0.973837,0.274218,97204.539640,281.624338,...,280.110530,283.971212,283.415766,-0.044240,0.035848,282.111950,0.410170,-5.969296,-3.057521,0.562019
4102,2.548999e+06,26.314614,5.949103,9.458913e+05,3.397262,2.748993,-4.893680,-5.514376,98426.428024,293.989830,...,292.406944,291.813822,289.469798,-0.672306,-0.699276,285.465405,0.410041,-5.992774,-6.548887,-6.869352
3044,2.502768e+06,15.676235,13.376008,1.767840e+06,3.400400,2.749808,3.114940,2.275835,97885.818838,286.329400,...,284.265488,288.395923,288.242009,0.248307,0.194265,285.991828,0.417008,-5.888201,5.406714,3.936695
762,2.544937e+06,13.567123,0.416816,4.004912e+05,3.226824,2.701542,-1.835877,-1.335212,97921.588850,291.210395,...,288.208851,295.079385,294.162724,-0.088700,-0.057908,290.528525,0.410140,-5.969064,-4.162370,-2.733329


In [4]:
print('The shape of the data table is:')
print('===============================')
print(X_train.shape)
print()

print('The types of the attributes are:')
print('================================')
X_train.info()

print()

print('How many missing values per attribute:')
print('======================================')
print(X_train.isnull().sum())

print()

print('Fraction of missing values per attribute:')
print('======================================')
print(X_train.isnull().mean())


The shape of the data table is:
(3181, 22)

The types of the attributes are:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3181 entries, 1435 to 329
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   p54.162.13  3181 non-null   float64
 1   p55.162.13  3181 non-null   float64
 2   cape.13     3181 non-null   float64
 3   p59.162.13  3181 non-null   float64
 4   lai_lv.13   3181 non-null   float64
 5   lai_hv.13   3181 non-null   float64
 6   u10n.13     3181 non-null   float64
 7   v10n.13     3181 non-null   float64
 8   sp.13       3181 non-null   float64
 9   stl1.13     3181 non-null   float64
 10  u10.13      3181 non-null   float64
 11  v10.13      3181 non-null   float64
 12  t2m.13      3181 non-null   float64
 13  stl2.13     3181 non-null   float64
 14  stl3.13     3181 non-null   float64
 15  iews.13     3181 non-null   float64
 16  inss.13     3181 non-null   float64
 17  stl4.13     3181 non-null   fl

Como podemos observar con la salida anterior, en nuestro conjunto de datos está compuesto por 3181 entradas y tenemos 22 vatiables, no existe ningún valor nulo y todos los datos son de tipo float64. 

A continuación estudiaremos si lso datos se encuentran equibrados